In [1]:
import json
from pydantic import BaseModel, Field
from typing import List, Optional
from geopy.distance import geodesic
from sklearn.cluster import KMeans
import numpy as np

# Define models for User, Vehicle, Event, Ride, and AssignedRide
class User(BaseModel):
    id: int
    firstName: str
    lastName: str
    email: str
    address: str
    city: str
    isSmoking: bool
    isTalkative: bool
    seatingPreference: bool
    preferredGenres: List[dict]  # Example: {"id": 1, "name": "Rock"}

class Vehicle(BaseModel):
    id: int
    brand: str
    model: str
    color: str
    plate: str
    maxPassengers: int

class Event(BaseModel):
    id: int
    title: str
    description: str
    startDate: str
    endDate: str
    address: str
    registerDeadline: str
    longitude: float
    latitude: float

class Ride(BaseModel):
    id: int
    eventId: int
    isDriver: bool
    userId: int
    vehicleId: Optional[int]
    pickupRadius: Optional[float]
    pickupLong: float
    pickupLat: float
    pickupSequence: Optional[int]
    driverId: Optional[int]
    can_be_driver: bool
    vehicle: Optional[Vehicle]
    user: User
    event: Event

class AssignedRide(Ride):
    assigned_driver: Optional[Ride] = None

# Load data from the provided JSON file
with open('example.json', 'r') as file:
    data = json.load(file)

rides = [Ride(**ride) for ride in data['rides']]

# Function to calculate optimized route for a driver, including pickup points
def calculate_route_with_pickup_points(driver_ride, assigned_rides):
    driver_location = (driver_ride.pickupLat, driver_ride.pickupLong)
    event_location = (driver_ride.event.latitude, driver_ride.event.longitude)

    # Collect passenger locations assigned to this driver
    passenger_locations = [
        (ride.pickupLat, ride.pickupLong) for ride in assigned_rides if ride.driverId == driver_ride.id
    ]

    # Identify pickup points using clustering if there are more than 3 stops (driver + passengers + event)
    pickup_points = []
    if len(passenger_locations) > 2:  # More than 3 stops means at least 2 passengers
        kmeans = KMeans(n_clusters=min(len(passenger_locations), 3), random_state=0).fit(passenger_locations)
        pickup_points = kmeans.cluster_centers_.tolist()
    else:
        pickup_points = passenger_locations

    # Create the route starting from the driver location
    all_locations = [driver_location] + pickup_points + [event_location]

    # Optimize the route (simple nearest neighbor)
    route = [all_locations[0]]
    remaining_points = all_locations[1:]

    while remaining_points:
        current = route[-1]
        next_point = min(remaining_points, key=lambda p: geodesic(current, p).meters)
        route.append(next_point)
        remaining_points.remove(next_point)

    return route, pickup_points

# Assign rides to drivers
assigned_rides = []
for ride in rides:
    if ride.isDriver:
        for other_ride in rides:
            if not other_ride.isDriver and other_ride.driverId == ride.id:
                assigned_rides.append(AssignedRide(**other_ride.dict(), assigned_driver=ride))

# Calculate and print optimized routes for each driver
for ride in rides:
    if ride.isDriver:
        driver_assigned_rides = [r for r in assigned_rides if r.assigned_driver.id == ride.id]
        optimized_route, pickup_points = calculate_route_with_pickup_points(ride, driver_assigned_rides)

        print(f"Optimized Route for Driver {ride.user.firstName} {ride.user.lastName}:")
        for i, stop in enumerate(optimized_route):
            print(f"  Stop {i + 1}: {stop}")

        if len(optimized_route) > 3:
            print("  Pickup Points:")
            for i, point in enumerate(pickup_points):
                print(f"    Pickup Point {i + 1}: {point}")
        print()


print("\nAssigned Rides:")
for assigned_ride in assigned_rides:
    print(f"Passenger: {assigned_ride.user.firstName} assigned to Driver: {assigned_ride.assigned_driver.user.firstName}")

Optimized Route for Driver John Doe:
  Stop 1: (50.8467, 4.3524)
  Stop 2: (50.8804, 4.4724)
  Stop 3: (51.2194, 4.4025)

Optimized Route for Driver Liam Gray:
  Stop 1: (51.215, 4.419)
  Stop 2: (51.2194, 4.4025)
  Stop 3: (51.2248, 4.4353)

Optimized Route for Driver Oliver Brown:
  Stop 1: (51.2196, 4.4377)
  Stop 2: (51.2194, 4.4025)

Optimized Route for Driver Lucas Van Dam:
  Stop 1: (51.0543, 3.7174)
  Stop 2: (51.0602, 3.7308)
  Stop 3: (51.2194, 4.4025)

Optimized Route for Driver Marie Lemmens:
  Stop 1: (51.1316, 4.5728)
  Stop 2: (51.1253, 4.5722)
  Stop 3: (51.2194, 4.4025)


Assigned Rides:
Passenger: Alice assigned to Driver: John
Passenger: Sophia assigned to Driver: Liam
Passenger: Sophie assigned to Driver: Lucas
Passenger: Eva assigned to Driver: Marie
